<a href="https://colab.research.google.com/github/0xstubbs/Rook-Governance/blob/main/Rook_Governance_Votes_by_Wallet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook takes a look at Rook Token holders over time. The purpose is to: 

1.   Determine token distribution over time.
2.   Determine quorum requirements



In [ ]:
import pandas as pd
import numpy as np
import altair as alt
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [200]:
!pip install altair_data_server

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Open csv with token balance data downloaded from Dune Analytics.

In [158]:
df = pd.DataFrame()
csvs = [  '/content/gdrive/MyDrive/Colab Notebooks/2020_token_holder_balances.csv'
        , '/content/gdrive/MyDrive/Colab Notebooks/21-01-01---21-05-31_token_holder_balances.csv'
        , '/content/gdrive/MyDrive/Colab Notebooks/21-06-01_to_21-12-31_rook_token_holder_balances.csv'
        , '/content/gdrive/MyDrive/Colab Notebooks/22-0101_to_22-0725_token_holders.csv'
        ]
for csv in csvs:
  df_new = pd.read_csv(csv)
  df = pd.concat([df, df_new], ignore_index = True)
  # df.append(df_new)

df['day'] = pd.to_datetime(df['day'])
df['rook_token_balance']=df['rook_token_balance'].fillna(0.0)
df['xrook_token_balance']=df['xrook_token_balance'].fillna(0.0)
df['total_balance']=df['total_balance'].fillna(0.0)
df=df.reset_index(drop=True)
df.info()
df.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752137 entries, 0 to 752136
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype              
---  ------               --------------   -----              
 0   day                  752137 non-null  datetime64[ns, UTC]
 1   address              752137 non-null  object             
 2   status               752137 non-null  object             
 3   rook_token_balance   752137 non-null  float64            
 4   xrook_token_balance  752137 non-null  float64            
 5   total_balance        752137 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(3), object(2)
memory usage: 34.4+ MB


,day,address,status,rook_token_balance,xrook_token_balance,total_balance
752132,2022-01-01 00:00:00+00:00,\x21008eb342c6ab9a4cceb7c9c497a0101ae31e3a,included,10.000820,0.0,10.000820
752133,2022-01-01 00:00:00+00:00,\x80ce59ec29c3b8837396323f0cf0f29a2ac0903d,included,10.000471,0.0,10.000471
752134,2022-01-01 00:00:00+00:00,\x8c9a359419b37520d87c9e55c04514a9b8549c72,included,10.000060,0.0,10.000060
752135,2022-01-01 00:00:00+00:00,\xd638dc334f3a9b6692f7aa71a32949f7da12f9d4,included,10.000042,0.0,10.000042
752136,2022-01-01 00:00:00+00:00,\xc2f064325b88960df3b6ae0522b93e6e27854e4c,included,10.000001,0.0,10.000001


Open csv containing 'Excluded' addresses.

In [199]:
df_excluded_addresses = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/22-0725_excluded_addresses.csv')
df_excluded_addresses.info()
df_excluded_addresses

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   label             26 non-null     object
 1   contract_address  26 non-null     object
 2   category          26 non-null     object
 3   status            26 non-null     object
dtypes: object(4)
memory usage: 960.0+ bytes


,label,contract_address,category,status
0,Rook Strategic Reserves,\xb81f5b9bd373b9d0df2e3191a01b8fa9b4d2832a,Rook,excluded
1,Rook Treasury,\x9a67f1940164d0318612b497e8e6038f902a00a4,Rook,excluded
2,Sablier V1,\xcd18eaa163733da39c232722cbc4e8940b1d8888,Sablier,excluded
3,Liquidity Pools,\x9bc9bdfe0843f44cf0878700501bb9142cdad2b8,Rook,excluded
4,Labs Multisig,\xa8b0a855be21568b93f32805b244158cc61ad006,Rook,excluded
5,Rook: Liquidity Pool v4,\x4f868c1aa37fcf307ab38d215382e88fca6275e2,Rook,excluded
6,Rook: Coordinator Staking,\x46168a99d15e6c620577d75ed7c453a6f53bafd5,Rook,excluded
7,Rook: Hiding Game Distributor,\xd81e97075dbda444ef65db3a96706c679b5311fd,Rook,excluded
8,Rook: HG 4 Distributor,\x25d27d026ab896fad8c8080f5b0e3fbd6667b139,Rook,excluded
9,Rook: LPQ2 Distributor,\x2777b798fdfb906d42b89cf8f9de541db05dd6a1,Rook,excluded


In [198]:
# df.head()

In [187]:
df_500 = df[(df['total_balance'] > 500) & (df['status']=='included')].copy()

In [178]:
# df_500 = df_500.reset_index(drop=True)
df_500['rook_token_balance']=df_500['rook_token_balance'].fillna(0.0)
df_500['xrook_token_balance']=df_500['xrook_token_balance'].fillna(0.0)
df_500['total_balance']=df_500['total_balance'].fillna(0.0)
df_500

,day,address,status,rook_token_balance,xrook_token_balance,total_balance
1,2020-12-31 00:00:00+00:00,\xf286bb612e219916f8e9ba7200bf09ed218890cb,included,100000.000000,0.0,100000.000000
2,2020-12-31 00:00:00+00:00,\xf55a73a366f1f9f03cef4cc10d3cd21e5c6a9026,included,51635.820509,0.0,51635.820509
3,2020-12-31 00:00:00+00:00,\xcadf6735144d1d7f1a875a5561555cba5df2f75c,included,28965.882114,0.0,28965.882114
4,2020-12-31 00:00:00+00:00,\xf30026fe8a2c0d01b70b1949ceaf2e09efd8b4a5,included,18000.000000,0.0,18000.000000
6,2020-12-31 00:00:00+00:00,\xdf73ff339a9250c553d518fee2666455bab0ed3f,included,10671.389461,0.0,10671.389461
...,...,...,...,...,...,...
750321,2022-01-01 00:00:00+00:00,\x195fca8382363a06eeb3ab93aaa71b3aa3a337ca,included,535.548066,0.0,535.548066
750322,2022-01-01 00:00:00+00:00,\xb8001c3ec9aa1985f6c747e25c28324e4a361ec1,included,517.703226,0.0,517.703226
750323,2022-01-01 00:00:00+00:00,\x8d9f950c23b73edf79ce52f74c6fb589cd2cbd90,included,513.091384,0.0,513.091384
750324,2022-01-01 00:00:00+00:00,\x80825bf6c668168c81cdde78bcea8d56dae92ed1,included,512.521838,0.0,512.521838


In [191]:
df_500_week = df_500.set_index('day').groupby([pd.Grouper(freq='W'), 'address']).mean()
df_500_week=df_500_week.reset_index()

In [196]:
# df_500_week.head()

In [195]:
# df_500[(df.day < '2021-01-01')].head(50)

In [197]:
alt.data_transformers.enable('data_server')
alt.renderers.enable('default')

alt.Chart(df_500_week).mark_area().encode(
    alt.X('day:T'),
    alt.Y('total_balance:Q'),
    alt.Color('address:N'),
    tooltip = [
               alt.Tooltip("address:N"),
               alt.Tooltip("day:T"),
               alt.Tooltip("rook_token_balance:Q", format=(",.0f")),
               alt.Tooltip("xrook_token_balance:Q", format=(",.0f")),
              #  alt.Tooltip("wallet_label:N"),
               alt.Tooltip("total_balance:Q", format=(",.0f"))
    ]
).properties(height=800, width=1000)


alt.Chart(...)